# Textual MOC Application with Aladin Lite

The example illustrates how to integrate and utilize the textual MOC within a web page using [Aladin Lite](https://aladin.cds.unistra.fr/AladinLite/). It begins by structuring the HTML to include essential elements like a container for the Aladin Lite viewer, where the MOC map is displayed, and a popup that provides information when the cursor hovers over the MOC region. The content information displayed in the popup is extracted from the textual part of the MOC. Additionally, there's a menu box with a button that allows users to toggle sound effects on or off, enhancing the interactive experience.

Users can then interact with the Aladin Lite viewer, exploring the map, viewing contextual information in the popup, and toggling sound effects to enhance their experience. This setup effectively combines visual, textual, and auditory elements to create an immersive and interactive way to explore data in a textual MOC data structure. 

In [ ]:
import os
import json
import threading
import socketserver
import http.server
import webbrowser
import socket

# Define the HTML content, referencing external CSS and JS files
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Aladin Lite with Popup and Context Menu</title>
    <!-- Include the external CSS file -->
    <link rel="stylesheet" href="css/styles.css">
    <!-- Include the Aladin Lite JavaScript library -->
    <script src="https://aladin.cds.unistra.fr/AladinLite/api/v3/latest/aladin.js" charset="utf-8"></script>
</head>
<body>



    <!-- Informational container explaining the functionality -->
    <div id="info-container">
        <h2>Textual MOC Application with Aladin Lite</h2>
        <p>
           This application demonstrates a simple example of using Textual MOCs in an interactive page powered by Aladin Lite.
           Hovering over the magenta-colored MOC will open a popup at the bottom displaying the information contained in the textual MOC.
           A button allows you to enable sounds that will be played when the cursor enters or exits the region defined by the MOC.
        </p>
    </div>
    <!-- Container for the Aladin Lite viewer -->
    <div id="aladin-lite-div"></div>

    <!-- Popup that shows information when the cursor is inside the MOC region -->
    <div id="popup">You are inside the MOC region!</div>

    <!-- Menu box with a button to toggle sound effects -->
    <div id="menu-box">
        <button id="sound-toggle">Enable Sounds</button>
    </div>

    <!-- Audio elements for sound effects -->
    <audio id="enter-sound" preload="auto">
        <source src="https://cdn.jsdelivr.net/gh/ggreco77/TextualMOC@main/AladinGame/sounds/enter.mp3" type="audio/mpeg">
        Your browser does not support the audio element.
    </audio>
    <audio id="exit-sound" preload="auto">
        <source src="https://cdn.jsdelivr.net/gh/ggreco77/TextualMOC@main/AladinGame/sounds/exit.mp3" type="audio/mpeg">
        Your browser does not support the audio element.
    </audio>

    <!-- Include the external JavaScript file -->
    <script src="js/script.js"></script>
</body>
</html>
"""

# Content of the CSS file
css_content = """
/* Basic styling for the body to center the content */
body {
    font-family: Arial, sans-serif;
    width: 100%;
    height: auto;
    min-height: 100%;
    overflow-x: hidden;   
    overflow-y: auto;     
    box-sizing: border-box;
}

/* Styling for the informational container */
#info-container {
    background-color: #ffffff;
    padding: 20px;
    margin: 0;
    border-radius: 0;
    box-shadow: none;
    max-width: 100%;
    width: 100%;
    box-sizing: border-box;
    text-align: center;
}

/* Styling for the Aladin Lite container */
#aladin-lite-div {
    width: 100%;
    height: 500px;
    margin: 0 auto;
    position: relative;
    background-color: #000;
}

/* Popup styling, initially hidden */
#popup {
    padding: 10px;
    background-color: rgba(0, 0, 0, 0.7);
    color: white;
    border-radius: 0;
    display: none; /* Hidden by default */
    text-align: center;
    width: 100%;
    position: absolute;
    bottom: 0;
    left: 0;
}

/* Styling for the menu box containing the sound toggle button */
#menu-box {
    width: 100%;
    margin: 0;
    text-align: center;
    padding: 10px;
    background-color: #ffffff;
    border-radius: 0;
    box-shadow: none;
}

/* Styling for the sound toggle button */
#sound-toggle {
    display: block;
    margin: 0 auto;
    padding: 10px 20px;
    background-color: #007bff;
    color: white;
    border: none;
    border-radius: 5px;
    cursor: pointer;
    text-align: center;
    font-size: 16px;
}




"""

# Content of the JavaScript file
javascript_content = """
let aladin;
let cursorInMOC = false; // Track if the cursor is inside the MOC region
let soundsEnabled = false; // Variable to control if sounds are enabled
let moc; // This will hold the MOC data without text
let mocText = ''; // This will hold the MOC text

const popup = document.getElementById('popup');
const aladinContainer = document.getElementById('aladin-lite-div');
const enterSound = document.getElementById('enter-sound');
const exitSound = document.getElementById('exit-sound');
const soundToggle = document.getElementById('sound-toggle');

// Function to process a MOC object containing text and other data
function processMOCWithText(mocWithText) {
    const mocData = {};
    // Extract text and data separately from the MOC object
    for (const key in mocWithText) {
        if (key === 'text') {
            mocText = mocWithText[key]; // Store the text separately
        } else {
            mocData[key] = mocWithText[key]; // Store the other data in mocData
        }
    }
    return mocData;
}

// Function to toggle sound effects on or off
function toggleSounds() {
    soundsEnabled = !soundsEnabled;
    soundToggle.textContent = soundsEnabled ? "Disable Sounds" : "Enable Sounds"; // Update button text
}

// Add event listener to toggle sound on button click
soundToggle.addEventListener('click', toggleSounds);

// Initialize the Aladin Lite viewer
A.init.then(() => {
    aladin = A.aladin('#aladin-lite-div', { target: 'LMC', fov: 35 }); // Create the Aladin instance

    // Load MOC data from an external JSON file
    fetch('moc/mocData.json')
    .then(response => response.json())
    .then(moc_with_text => {
        // Process the MOC to separate area data and text
        const mocData = processMOCWithText(moc_with_text);

        // Create the MOC object for Aladin with specified appearance settings
        moc = A.MOCFromJSON(mocData, { opacity: 0.45, color: 'magenta', lineWidth: 1, fill: true, perimeter: true});
        aladin.addMOC(moc);

        // Handle mouse movement events to detect when the cursor enters or leaves the MOC region
        aladin.on('mouseMove', function (pos) {
            if (!pos) return; // If no position data is available, exit the function

            // Check if the cursor is within the MOC region
            let isInMOC = moc.contains(pos.ra, pos.dec);

            if (isInMOC && !cursorInMOC) {
                // When entering the MOC region
                console.log('Cursor entered the MOC region');
                cursorInMOC = true; // Update state
                popup.style.display = 'block'; // Show the popup
                popup.textContent = mocText; // Update the popup with the MOC text

                // Play entry sound if sounds are enabled
                if (soundsEnabled) {
                    enterSound.play().catch(error => console.log("Error playing entry sound: ", error));
                }
            } else if (!isInMOC && cursorInMOC) {
                // When leaving the MOC region
                console.log('Cursor left the MOC region');
                cursorInMOC = false; // Update state
                popup.style.display = 'none'; // Hide the popup

                // Play exit sound if sounds are enabled
                if (soundsEnabled) {
                    exitSound.play().catch(error => console.log("Error playing exit sound: ", error));
                }
            }
        });
    });
});
"""

# MOC data separated into a JSON file
moc_data = {
    "5": [8263, 8274],
    "6": [33047, 33049, 33050, 33051, 33072, 33073, 33076, 33077, 33078, 33090, 33091, 33094, 33095, 33100, 33101, 33102, 33106, 33120, 33121],
    "7": [132186, 132187, 132195, 132245, 132297, 132300, 132301, 132303, 132316, 132317, 132318, 132354, 132355, 132358, 132359, 132370, 132371, 132374, 132375, 132412, 132413, 132414, 132418, 132448, 132449, 132450, 132488, 132489, 132496, 132497],
    "8": [528631, 528637, 528638, 528639, 528699, 528701, 528702, 528703, 528731, 528734, 528735, 528739, 528741, 528742, 528743, 528775, 528779, 528988, 528989, 528991, 529184, 529185, 529187, 529196, 529197, 529208, 529209, 529211, 529276, 529277, 529278, 529426, 529427, 529430, 529431, 529474, 529475, 529478, 529479, 529490, 529491, 529494, 529495, 529660, 529661, 529662, 529666, 529676, 529678, 529679, 529712, 529714, 529720, 529722, 529804, 529805, 529806, 529808, 529824, 529825, 529826, 529960, 529961, 529968, 529969, 529970, 529972, 529992, 529993, 529994, 529996, 530000, 530001, 530002],
    "9": [2114517, 2114519, 2114523, 2114545, 2114547, 2114783, 2114795, 2114801, 2114802, 2114803, 2114922, 2114923, 2114931, 2114934, 2114935, 2114951, 2114953, 2114954, 2114955, 2114961, 2114962, 2114963, 2115093, 2115094, 2115095, 2115097, 2115098, 2115099, 2115109, 2115110, 2115111, 2115113, 2115114, 2115115, 2115961, 2115963, 2116052, 2116053, 2116055, 2116744, 2116745, 2116747, 2116772, 2116773, 2116796, 2116797, 2116840, 2116841, 2116843, 2117116, 2117117, 2117204, 2117205, 2117376, 2117377, 2117380, 2117381, 2117392, 2117393, 2117396, 2117658, 2117659, 2117662, 2117663, 2117718, 2117719, 2117890, 2117891, 2117894, 2117895, 2117906, 2117907, 2117910, 2117911, 2117954, 2117955, 2117958, 2117959, 2117970, 2117971, 2117974, 2118652, 2118653, 2118654, 2118668, 2118670, 2118671, 2118682, 2118708, 2118710, 2118711, 2118760, 2118762, 2118763, 2118854, 2118860, 2118862, 2118884, 2118886, 2118892, 2118894, 2119228, 2119230, 2119240, 2119241, 2119242, 2119264, 2119308, 2119309, 2119310, 2119312, 2119313, 2119314, 2119328, 2119329, 2119330, 2119848, 2119849, 2119856, 2119857, 2119858, 2119860, 2119861, 2119884, 2119885, 2119886, 2119892, 2119893, 2119894, 2119896, 2119980, 2119981, 2119982, 2119988, 2119989, 2119992, 2120012, 2120016, 2120017, 2120064],
    "10": [8457725, 8457727, 8458067, 8458073, 8458074, 8458075, 8458085, 8458087, 8458173, 8458175, 8458177, 8458179, 8458185, 8458186, 8458187, 8459130, 8459131, 8459163, 8459165, 8459166, 8459167, 8459174, 8459175, 8459177, 8459178, 8459179, 8459203, 8459515, 8459517, 8459518, 8459519, 8459679, 8459686, 8459687, 8459722, 8459723, 8459731, 8459734, 8459735, 8459802, 8459803, 8459810, 8459811, 8459842, 8459843, 8460349, 8460350, 8460351, 8460366, 8460367, 8460371, 8460385, 8460386, 8460387, 8460429, 8460430, 8460431, 8460433, 8460434, 8460435, 8460449, 8460450, 8460451, 8463637, 8463639, 8463645, 8463647, 8463669, 8463671, 8463677, 8463679, 8463765, 8463840, 8463841, 8463843, 8463849, 8464197, 8464199, 8464216, 8464217, 8464219, 8464244, 8464245, 8464247, 8466984, 8466985, 8466987, 8467076, 8467077, 8467097, 8467100, 8467101, 8467103, 8467169, 8467172, 8467173, 8467175, 8467196, 8467197, 8467368, 8467369, 8467371, 8468472, 8468473, 8468476, 8468477, 8468753, 8468756, 8468757, 8468800, 8468801, 8468804, 8468805, 8469588, 8469589, 8469590, 8469760, 8469761, 8469764, 8469765, 8469776, 8469777, 8469780, 8470586, 8470587, 8470590, 8470591, 8470642, 8470643, 8470646, 8470647, 8470811, 8470814, 8470815, 8470858, 8470859, 8470862, 8470863, 8471570, 8471571, 8471574, 8471575, 8471618, 8471619, 8471622, 8471623, 8471634, 8471635, 8471638, 8471639, 8471810, 8471811, 8471814, 8471815, 8471826, 8471827, 8471830, 8471831, 8471900, 8471902, 8471903, 8474620, 8474634, 8474635, 8474638, 8474678, 8474679, 8474722, 8474732, 8474734, 8474735, 8474746, 8474838, 8474839, 8475016, 8475018, 8475019, 8475044, 8475046, 8475066, 8475408, 8475410, 8475411, 8475444, 8475446, 8475452, 8475454, 8475540, 8475542, 8475548, 8475550, 8475572, 8476916, 8476917, 8476918, 8476944, 8476945, 8476946, 8476952, 8476954, 8476972, 8476974, 8477064, 8477244, 8477245, 8477246, 8477260, 8477264, 8477280, 8477282, 8477324, 8477325, 8477326, 8477328, 8477329, 8477330, 8477344, 8477345, 8479400, 8479408, 8479409, 8479412, 8479413, 8479436, 8479437, 8479438, 8479448, 8479449, 8479548, 8479549, 8479580, 8479581, 8479582, 8479588, 8479589, 8479590, 8479600, 8479616, 8479617, 8479618, 8479620, 8479932, 8479933, 8479934, 8479960, 8479961, 8479962, 8479964, 8479972, 8479973, 8480052, 8480053, 8480056, 8480072, 8480073, 8480074, 8480080, 8480128, 8480129, 8480130, 8480260, 8480261, 8480262, 8480264, 8480272],
    "text": "Nearly 200,000 light-years from Earth, the Large Magellanic Cloud, a satellite galaxy of the Milky Way," \
    " floats in space, in a long and slow dance around our galaxy. Vast clouds of gas within it slowly collapse to form" \
    " new stars. In turn, these light up the gas clouds in a riot of colors," \
    " visible in this image from the NASA/ESA Hubble Space Telescope" \
    " https://science.nasa.gov/missions/hubble/large-magellanic-cloud/"
}

# Create the main directory 'aladin_game_d1' if it doesn't exist
base_dir = 'aladin_game_d1'
os.makedirs(os.path.join(base_dir, 'css'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'js'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'moc'), exist_ok=True)

# Write the HTML content to 'index.html' in the 'aladin_game_d1' folder
with open(os.path.join(base_dir, 'index.html'), 'w') as file:
    file.write(html_content)

# Write the CSS content to 'styles.css' in the 'css' folder
with open(os.path.join(base_dir, 'css', 'styles.css'), 'w') as file:
    file.write(css_content)

# Write the JavaScript content to 'script.js' in the 'js' folder
with open(os.path.join(base_dir, 'js', 'script.js'), 'w') as file:
    file.write(javascript_content)

# Write the MOC data to 'mocData.json' in the 'moc' folder
with open(os.path.join(base_dir, 'moc', 'mocData.json'), 'w') as file:
    json.dump(moc_data, file)

# Function to find an available port
def find_free_port():
    with socket.socket() as s:
        s.bind(('', 0))
        return s.getsockname()[1]

# Function to start the server and open the browser
def start_server_and_open_browser():
    # Change the directory to 'aladin_game_d1' where the HTML file is located
    os.chdir(os.path.abspath(base_dir))

    # Handler to serve files
    Handler = http.server.SimpleHTTPRequestHandler

    # Find a free port
    port = find_free_port()

    with socketserver.TCPServer(("", port), Handler) as httpd:
        # Open the page in the default web browser
        webbrowser.open(f"http://localhost:{port}/index.html")
        print(f"Serving at port {port}")
        httpd.serve_forever()

# Run the server in a separate thread to avoid blocking
server_thread = threading.Thread(target=start_server_and_open_browser)
server_thread.daemon = True
server_thread.start()

# Keep the main thread alive to allow the server to run
try:
    while True:
        pass
except KeyboardInterrupt:
    print("Shutting down the server.")

Serving at port 51825


127.0.0.1 - - [17/Sep/2025 13:26:19] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 13:26:19] "GET /css/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 13:26:19] "GET /js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 13:26:20] "GET /moc/mocData.json HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2025 13:26:20] code 404, message File not found
127.0.0.1 - - [17/Sep/2025 13:26:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Sep/2025 13:26:36] code 404, message File not found
127.0.0.1 - - [17/Sep/2025 13:26:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Sep/2025 13:26:41] "GET /css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2025 13:26:41] code 404, message File not found
127.0.0.1 - - [17/Sep/2025 13:26:41] "GET /.well-known/appspecific/com.chrome.devtools.json HTTP/1.1" 404 -
127.0.0.1 - - [17/Sep/2025 13:27:07] "GET /index.html HTTP/1.1" 304 -
127.0.0.1 - - [17/Sep/2025 13:27:07] code 404, message File not found
127.0.0.1 - - [17/Sep/2025 13:27:0